<a href="https://colab.research.google.com/github/FernandoSimon22/vision_artificial/blob/main/YOLOv5_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Clone GitHub repository, install dependencies and check PyTorch and GPU.

In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt comet_ml  # install
%pip install -q roboflow

import torch
import utils
import os
from IPython.display import Image, clear_output
display = utils.notebook_init()  # checks


YOLOv5 🚀 v7.0-418-ga493afe1 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)


Setup complete ✅ (8 CPUs, 51.0 GB RAM, 41.7/235.7 GB disk)


In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="lX9sE2RPQOCv2R7cn0io")
project = rf.workspace("jota22").project("am_boundingbox")
version = project.version(14)
dataset = version.download("yolov5")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to AM_BoundingBox-14 in yolov5pytorch:: 100%|██████████| 392/392 [00:00<00:00, 8868.79it/s]


In [3]:
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

### Train YOLOv5s

In [4]:
# Train YOLOv5s from scracht for 300 epochs
!python train.py --img 1024 --batch 4 --epochs 300 --data /content/yolov5/AM_BoundingBox-14/data.yaml --weights yolov5s.pt --cache

Se han truncado las últimas 5000 líneas del flujo de salida.
  with torch.cuda.amp.autocast(amp):
    177/299      3.02G    0.04401     0.2573          0        357       1024:  68% 23/34 [00:02<00:01,  7.73it/s]/content/yolov5/train.py:413: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
    177/299      3.02G    0.04392     0.2597          0        365       1024:  71% 24/34 [00:03<00:01,  7.76it/s]/content/yolov5/train.py:413: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
    177/299      3.02G    0.04382     0.2569          0        175       1024:  74% 25/34 [00:03<00:01,  7.77it/s]/content/yolov5/train.py:413: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autoc

# División en 5 folds

In [6]:
import os
import shutil
import random
from sklearn.model_selection import KFold
from pathlib import Path

# Configura tu ruta de dataset aquí
DATASET_PATH = "/content/yolov5/AM_BoundingBox-14/train"
IMAGES_DIR = os.path.join(DATASET_PATH, "images")
LABELS_DIR = os.path.join(DATASET_PATH, "labels")

# Número de folds
K_FOLDS = 5

# Lista de todas las imágenes y sus etiquetas
images = sorted([f for f in os.listdir(IMAGES_DIR) if f.endswith(".jpg") or f.endswith(".png")])
labels = sorted([f for f in os.listdir(LABELS_DIR) if f.endswith(".txt")])

# Comprobación
assert len(images) == len(labels), "Número de imágenes y etiquetas no coincide"

# Crear los folds
kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(images)):
    print(f"Preparando Fold {fold + 1}")

    fold_dir = f"/content/folds/fold{fold + 1}"
    train_img_dir = os.path.join(fold_dir, "train/images")
    train_lbl_dir = os.path.join(fold_dir, "train/labels")
    val_img_dir = os.path.join(fold_dir, "val/images")
    val_lbl_dir = os.path.join(fold_dir, "val/labels")

    os.makedirs(train_img_dir, exist_ok=True)
    os.makedirs(train_lbl_dir, exist_ok=True)
    os.makedirs(val_img_dir, exist_ok=True)
    os.makedirs(val_lbl_dir, exist_ok=True)

    for i in train_idx:
        shutil.copy(os.path.join(IMAGES_DIR, images[i]), os.path.join(train_img_dir, images[i]))
        shutil.copy(os.path.join(LABELS_DIR, labels[i]), os.path.join(train_lbl_dir, labels[i]))

    for i in val_idx:
        shutil.copy(os.path.join(IMAGES_DIR, images[i]), os.path.join(val_img_dir, images[i]))
        shutil.copy(os.path.join(LABELS_DIR, labels[i]), os.path.join(val_lbl_dir, labels[i]))

print("✅ División en 5 folds completada")

Preparando Fold 1
Preparando Fold 2
Preparando Fold 3
Preparando Fold 4
Preparando Fold 5
✅ División en 5 folds completada


# Validación cruzada (5-fold CV) con YOLOv5 y calcular las métricas promedio.

# ✅ 1. Configuración inicial

In [7]:
import os
import shutil
from sklearn.model_selection import KFold
import glob
from pathlib import Path
import yaml

# Rutas
IMAGES_DIR = '/content/yolov5/AM_BoundingBox-14/train/images'
LABELS_DIR = '/content/yolov5/AM_BoundingBox-14/train/labels'
FOLDS_DIR = '/content/folds'
os.makedirs(FOLDS_DIR, exist_ok=True)

# Archivos
image_files = sorted(glob.glob(os.path.join(IMAGES_DIR, '*.jpg')))
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# ✅ 2. Crear carpetas para cada fold

In [8]:
for fold, (train_idx, val_idx) in enumerate(kf.split(image_files), 1):
    fold_path = os.path.join(FOLDS_DIR, f'fold{fold}')
    train_img = os.path.join(fold_path, 'images', 'train')
    val_img = os.path.join(fold_path, 'images', 'val')
    train_lbl = os.path.join(fold_path, 'labels', 'train')
    val_lbl = os.path.join(fold_path, 'labels', 'val')
    for d in [train_img, val_img, train_lbl, val_lbl]: os.makedirs(d, exist_ok=True)

    for i in train_idx:
        img = image_files[i]
        lbl = os.path.join(LABELS_DIR, Path(img).stem + '.txt')
        shutil.copy(img, train_img)
        shutil.copy(lbl, train_lbl)

    for i in val_idx:
        img = image_files[i]
        lbl = os.path.join(LABELS_DIR, Path(img).stem + '.txt')
        shutil.copy(img, val_img)
        shutil.copy(lbl, val_lbl)

# ✅ 3. Ejecutar entrenamiento en cada fold

In [9]:
import subprocess

results = []

for fold in range(1, 6):
    fold_path = f'{FOLDS_DIR}/fold{fold}'

    # Crear YAML
    data_yaml = {
        'train': os.path.join(fold_path, 'images', 'train'),
        'val': os.path.join(fold_path, 'images', 'val'),
        'nc': 1,
        'names': ['def']
    }
    yaml_path = os.path.join(fold_path, 'data.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(data_yaml, f)

    print(f'🔁 Entrenando fold {fold}...')

    result = subprocess.run([
        'python', 'train.py',
        '--img', '640',
        '--batch', '16',
        '--epochs', '300',
        '--data', yaml_path,
        '--weights', 'yolov5s.pt',
        '--project', '/content/folds/results',
        '--name', f'fold{fold}',
        '--exist-ok'
    ])
    results.append(result)

🔁 Entrenando fold 1...
🔁 Entrenando fold 2...
🔁 Entrenando fold 3...
🔁 Entrenando fold 4...
🔁 Entrenando fold 5...


# ✅ 4. Promediar métricas de rendimiento (mAP, precisión, recall)

In [15]:
import pandas as pd

df = pd.read_csv('/content/folds/results/fold1/results.csv')
print(df.columns.tolist())


['               epoch', '      train/box_loss', '      train/obj_loss', '      train/cls_loss', '   metrics/precision', '      metrics/recall', '     metrics/mAP_0.5', 'metrics/mAP_0.5:0.95', '        val/box_loss', '        val/obj_loss', '        val/cls_loss', '               x/lr0', '               x/lr1', '               x/lr2']


In [16]:
import pandas as pd

maps = []
precisions = []
recalls = []

for fold in range(1, 6):
    path = f'/content/folds/results/fold{fold}/results.csv'
    df = pd.read_csv(path)
    last_row = df.iloc[-1]

    # Asignación segura por nombre parcial de columnas
    mAP_col = [col for col in df.columns if 'mAP_0.5' in col][0]
    prec_col = [col for col in df.columns if 'precision' in col][0]
    recall_col = [col for col in df.columns if 'recall' in col][0]

    maps.append(float(last_row[mAP_col]))
    precisions.append(float(last_row[prec_col]))
    recalls.append(float(last_row[recall_col]))

print(f'📊 Promedio mAP@0.5: {sum(maps)/5:.4f}')
print(f'📊 Promedio precisión: {sum(precisions)/5:.4f}')
print(f'📊 Promedio recall: {sum(recalls)/5:.4f}')


📊 Promedio mAP@0.5: 0.5796
📊 Promedio precisión: 0.6254
📊 Promedio recall: 0.5823


In [17]:
# YOLOv5 PyTorch HUB Inference (DetectionModels only)
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True, trust_repo=True)  # or yolov5n - yolov5x6 or custom
im = 'https://ultralytics.com/images/zidane.jpg'  # file, Path, PIL.Image, OpenCV, nparray, list
results = model(im)  # inference
results.print()  # or .show(), .save(), .crop(), .pandas(), etc.

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 v7.0-418-ga493afe1 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
image 1/1: 720x1280 2 persons, 1 tie, 1 cell phone
Speed: 1054.5ms pre-process, 43.6ms inference, 140.4ms NMS per image at shape (1, 3, 384, 640)
